In [1]:
%load_ext cython

In [8]:
%%cython -+ 
cimport cython
import numpy as np
cimport numpy as np

from libc.math cimport exp, tanh
from mc_lib.rndm cimport RndmWrapper
from mc_lib.lattices import tabulate_neighbors
from mc_lib.observable cimport RealObservable


cdef void init_spins(long[::1] spins, RndmWrapper rndm): 
    for j in range(spins.shape[0]):
        spins[j] = 1 if rndm.uniform() > 0.5 else -1
        
        
@cython.boundscheck(False)
@cython.wraparound(False)
cdef double energy(long[::1] spins, 
                   long[:, ::1] neighbors,
                  const double[:,::1] Js):

    cdef:
        double ene = 0.0
        Py_ssize_t site, site1, num_neighb

    for site in range(spins.shape[0]):
        num_neighb = neighbors[site, 0]
        for j in range(1, num_neighb+1):
            site1 = neighbors[site, j]
            ene += -1 * Js[site, site1] * spins[site] * spins[site1] 
    
    return ene / 2.0


@cython.boundscheck(False)
@cython.wraparound(False)
cdef void flip_spin(long[::1] spins, 
                    const long[:, ::1] neighbors,
                    double beta,
                    const double[:,::1] Js, RndmWrapper rndm): 
    cdef:
        Py_ssize_t site = int(spins.shape[0] * rndm.uniform())
        Py_ssize_t site1

    cdef long num_neighb = neighbors[site, 0]
    cdef double summ = 0.
    for j in range(1, num_neighb + 1):
        site1 = neighbors[site, j]
        summ += spins[site1] * spins[site] * Js[site,site1]
   
    cdef double ratio = exp(-2.0 * beta * summ )
    
    if rndm.uniform() > ratio:
        return

    spins[site] = -spins[site]
    
    
cdef void get_J( double[:,::1] Js, double J, double Jd, int L1, int L2):
    cdef Py_ssize_t i
    for i in range(L1*L2):
        Js[i, ((i // L2 + 1) % L1 * L2 )  + (i + 1) % L2 ] = Jd
        Js[i, ((i // L2  - 1) % L1 * L2 )  + (i - 1) % L2 ] = Jd
        Js[i, (i // L2) * L2 + (i + 1) % L2] = J 
        Js[i, (i + L2) % (L1*L2)] = J
        Js[i, (i // L2) * L2 + (i - 1) % L2] = J
        Js[i, (i - L2) % (L1*L2)] = J
    return
        

def simulate(Py_ssize_t L,
             double T, double J, double Jd,
             Py_ssize_t num_sweeps, int seed, int rseed = 1234):

    cdef:
        long[:, ::1] neighbors = tabulate_neighbors(L, kind='triang') 
        double beta = 1./T

    cdef:
        int num_therm = int(30 * L)
        int steps_per_sweep = L * L 
        int sweep = 0
        int i
        double Z = 0., magn = 0., binder = 0., error = 0.
        
    
    cdef RndmWrapper rndm = RndmWrapper((rseed, seed)) 
    cdef RealObservable m2 = RealObservable()
    cdef RealObservable m4 = RealObservable()

    cdef long[::1] spins =  np.empty( L*L, dtype=int) 
    init_spins(spins, rndm)
    
    cdef double[:,::1] Js = np.zeros((L*L, L*L)) 
    get_J(Js, J, Jd, L, L)

    for sweep in range(num_therm):
        for i in range(steps_per_sweep):
            flip_spin(spins, neighbors, beta, Js, rndm)

    m = np.zeros(num_sweeps)

    for sweep in range(num_sweeps):
        for i in range(steps_per_sweep):
            flip_spin(spins, neighbors, beta, Js, rndm)
            
        Z += 1
        magn = 0.
        for i in range(L*L):
            magn += spins[i]
            
        m2.add_measurement(magn**2)
        m4.add_measurement(magn**4)
        m[sweep] = magn
        
    binder = 1 - (m4.mean) / (3 * (m2.mean**2))
    
    error = np.sqrt( ( m4.errorbar/( 3*(m2.mean**2)) )**2 + ( 2*(m4.mean)*m2.errorbar/(3*(m2.mean**3) ) )**2 )
     
    return (binder)#, error, m)

In file included from /home/br/.virtualenvs/mc_lib/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944,
                 from /home/br/.virtualenvs/mc_lib/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/br/.virtualenvs/mc_lib/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/br/.cache/ipython/cython/_cython_magic_561f27a461ac91cfe28109996b1d1d50.cpp:652:
/home/br/.virtualenvs/mc_lib/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
/home/br/.cache/ipython/cython/_cython_magic_561f27a461ac91cfe28109996b1d1d50.cpp:2848:15: warning: ‘double __pyx_f_46_cython_magic_561f27a461ac91cfe28109996b1d1d50_energy(__Pyx_memviewslice, __Pyx_me

In [9]:
simulate(L=4, T=5, J=1, Jd=1, num_sweeps=10, seed=0)

0.5110854799436115